In [2]:
from Preprocessing.data_clean import remove_221_rows, dropping_no_betting_data, josh_features, fill_f_pm_01m, class_or_rating_average, oli_features
from v2_preprocessor import preprocess_features_v2
import pandas as pd

data  = pd.read_csv("../raw_data/raw_data_v2.2.csv")

data_cleaned = remove_221_rows(data)
data_cleaned = dropping_no_betting_data(data_cleaned)
data_cleaned = josh_features(data_cleaned)
data_cleaned = class_or_rating_average(data_cleaned)
data_cleaned = oli_features(data_cleaned)
data_cleaned = fill_f_pm_01m(data_cleaned)



preprocessed_data = preprocess_features_v2(data_cleaned)

preprocessed_data.to_csv("../raw_data/data_cleaned_and_preprocessed.csv", index=False)

data = pd.read_csv("../raw_data/data_cleaned_and_preprocessed.csv")
data.sort_values(by='f_ko')
data = data[(data['f_pm_01m'] <= 25)]
data = data.reset_index(drop=True)
print(data.shape)

ModuleNotFoundError: No module named 'v2_preprocessor'

In [ ]:
backtest = data[['f_ko','f_id', 'id','f_horse','f_pm_01m', 'linear_target', 'f_place']]

# #X = data[['f_runners','or_rating_vs_avg_race', 'stall_position', 
#           'iv_horse_at', 'iv_trainer_l200r', 'iv_trainer_l50r', 'iv_trainer_l16r', 'iv_trainer_at',
#           'iv_jockey_l200r', 'iv_jockey_l50r', 'iv_jockey_l16r', 'iv_jockey_at',
#           'ae_horse_l10r', 'ae_horse_l5r' , 'ae_horse_l2r' ,'ae_horse_at',
#           'ae_trainer_l200r' , 'ae_trainer_l50r' , 'ae_trainer_l16r' , 'ae_trainer_at',
#           'ae_jockey_l200r' , 'ae_jockey_l50r', 'ae_jockey_l16r' , 'ae_jockey_at' , 'f_dob', 'f_prb_avg', 'pred_isp']]



X = data[['stall_position', 
          'iv_trainer_l200r', 'iv_jockey_l200r',
          'ae_trainer_l200r' ,'ae_jockey_l200r', 
          'iv_trainer_at', 'iv_jockey_at',
          'ae_trainer_at' ,'ae_jockey_at',
          'iv_trainer_l50r', 'iv_jockey_l50r',
          'ae_trainer_l50r' ,'ae_jockey_l50r',
          'iv_trainer_l16r', 'iv_jockey_l16r',
          'ae_trainer_l16r' ,'ae_jockey_l16r','pred_isp']]







y = data["f_place"] #OR 'linear_target'

print(data.shape)
print(X.shape)
print(y.shape)
print(backtest.shape)

#Train = Year 1
#Val = Year 2
#Test = Year 3 (6 months)


In [ ]:
X_train = X.iloc[:33243]
X_val = X.iloc[33243:66406]
X_test = X.iloc[66406:]
y_train = y.iloc[:33243]
y_val = y.iloc[33243:66406]
y_test = y.iloc[66406:]
backtest_train = backtest.iloc[:33243]
backtest_val = backtest.iloc[33243:66406]
backtest_test = backtest.iloc[66406:]

In [ ]:
X_val.shape

In [ ]:
#buggy data = 2532754    2021-06-20 03:55:00
                #29406    20-06-14 18:45:00

In [ ]:
# convert labels to a one-hot vector
# from tensorflow.keras.utils import to_categorical

# train_labels = to_categorical(train_labels)
# test_labels = to_categorical(test_labels)

# define network architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import Precision, Accuracy, Recall, F1Score
from tensorflow.keras.losses import Poisson
from tensorflow.keras.optimizers import Adam

MLP2 = Sequential()
MLP2.add(InputLayer(input_shape=(18, ))) # input layer
MLP2.add(Dense(32, activation='relu')) # hidden layer 1
MLP2.add(Dense(1, activation='sigmoid')) # output layer


In [ ]:
# optimization
MLP2.compile(loss='binary_focal_crossentropy',
            optimizer=Adam(learning_rate = 0.0001),
            metrics=['squared_hinge'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# train (fit)

es = EarlyStopping(patience=5, restore_best_weights=True)

history = MLP2.fit(X_train, y_train, 
                  epochs=200, batch_size=32, verbose=1,
                  validation_data=(X_val, y_val), callbacks = [es], shuffle = True)

In [ ]:
# evaluate performance on test data
a = MLP2.evaluate(X_test, y_test,
                                         verbose=1)
a

In [ ]:
y_pred = MLP2.predict(X_test)

In [ ]:
y_pred

In [ ]:
import seaborn as sns
sns.histplot(y_pred, bins = 50, kde=True)

In [ ]:
backtest_test['model_preds'] = y_pred
backtest_test['odds_preds'] = 1/ backtest_test['f_pm_01m']
backtest_test['model_edge_basic'] = backtest_test['model_preds'] - backtest_test['odds_preds']
backtest_test['model_edge_proportionate'] = backtest_test['model_preds'] / backtest_test['odds_preds']

In [ ]:
#backtest_test = backtest_test.drop(columns=['f_ko'])
model_preds_race = backtest_test.groupby('f_id').sum().reset_index()
model_preds_race2 = model_preds_race[['f_id', 'model_preds', 'odds_preds']]
dict_odds_preds = model_preds_race2.set_index('f_id')['odds_preds'].to_dict()
dict_race_preds = model_preds_race2.set_index('f_id')['model_preds'].to_dict()
backtest_test['race_model_preds'] = backtest_test['f_id'].map(dict_race_preds)
backtest_test['race_odds_preds'] = backtest_test['f_id'].map(dict_odds_preds)
backtest_test['model_preds_scaled'] = (backtest_test['model_preds'] / backtest_test['race_model_preds']) * backtest_test['race_odds_preds']
backtest_test['model_edge_basic_scaled'] = backtest_test['model_preds_scaled'] - backtest_test['odds_preds']
backtest_test['model_edge_prop_scaled'] = backtest_test['model_preds_scaled'] / backtest_test['odds_preds']

In [ ]:
# Create dataframes of horses above specific proba_difference thresholds

all_bets = backtest_test
above0 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0]
above5 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0.025]
above10 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0.05]
above15 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0.075]
above20 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0.10]
above30 = backtest_test[backtest_test['model_edge_basic_scaled'] >= 0.20]

# Calculate evolutions of profits for each threshold

dfall = all_bets
dfall['cumulative_profit'] = dfall['linear_target'].cumsum()

df0 = above0
df0['cumulative_profit'] = df0['linear_target'].cumsum()

df5 = above5
df5['cumulative_profit'] = df5['linear_target'].cumsum()

df10 = above10
df10['cumulative_profit'] = df10['linear_target'].cumsum()

df15 = above15
df15['cumulative_profit'] = df15['linear_target'].cumsum()

df20 = above20
df20['cumulative_profit'] = df20['linear_target'].cumsum()

df30 = above30
df30['cumulative_profit'] = df30['linear_target'].cumsum()

In [ ]:
# Create dataframes of horses above specific proba_difference thresholds

all_bets = backtest_test
above0b = backtest_test[backtest_test['model_edge_basic'] >= 0]
above5b = backtest_test[backtest_test['model_edge_basic'] >= 0.05]
above10b = backtest_test[backtest_test['model_edge_basic'] >= 0.1]
above15b = backtest_test[backtest_test['model_edge_basic'] >= 0.15]
above20b = backtest_test[backtest_test['model_edge_basic'] >= 0.20]
above30b = backtest_test[backtest_test['model_edge_basic'] >= 0.30]

# Calculate evolutions of profits for each threshold

dfall = all_bets
dfall['cumulative_profit'] = dfall['linear_target'].cumsum()

df0b = above0b
df0b['cumulative_profit'] = df0b['linear_target'].cumsum()

df5b = above5b
df5b['cumulative_profit'] = df5b['linear_target'].cumsum()

df10b = above10b
df10b['cumulative_profit'] = df10b['linear_target'].cumsum()

df15b = above15b
df15b['cumulative_profit'] = df15b['linear_target'].cumsum()

df20b = above20b
df20b['cumulative_profit'] = df20b['linear_target'].cumsum()

df30b = above30b
df30b['cumulative_profit'] = df30b['linear_target'].cumsum()

In [ ]:
import matplotlib.pyplot as plt
# Plot
plt.figure(figsize=(10,6))
#plt.plot(range(1, len(dfall) + 1), dfall['cumulative_profit'], label='all bets')
#plt.plot(range(1, len(df0) + 1), df0['cumulative_profit'], label='0 threshold')
plt.plot(range(1, len(df5) + 1), df5['cumulative_profit'], label='2.5 threshold')
plt.plot(range(1, len(df10) + 1), df10['cumulative_profit'], label='5 threshold')
plt.plot(range(1, len(df15) + 1), df15['cumulative_profit'], label='7.5 threshold')
plt.plot(range(1, len(df20) + 1), df20['cumulative_profit'], label='10 threshold')
plt.plot(range(1, len(df30) + 1), df30['cumulative_profit'], label='20 threshold')
#plt.plot(range(1, len(df5b) + 1), df5b['cumulative_profit'], label='5 threshold unscaled')
#plt.plot(range(1, len(df10b) + 1), df10b['cumulative_profit'], label='10 threshold unscaled')
#plt.plot(range(1, len(df15b) + 1), df15b['cumulative_profit'], label='15 threshold unscaled')
#plt.plot(range(1, len(df20b) + 1), df20b['cumulative_profit'], label='20 threshold unscaled')
#plt.plot(range(1, len(df30b) + 1), df30b['cumulative_profit'], label='30 threshold unscaled')

plt.title("xxx")
plt.xlabel("number of bets")
plt.ylabel("profit (£)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# åplt.hist(backtest_test['model_edge_proportionate'])

In [ ]:
# # Create dataframes of horses above specific proba_difference thresholds

# all_bets = backtest_test
# above0c = backtest_test[backtest_test['model_edge_proportionate'] >= 1]
# above5c = backtest_test[backtest_test['model_edge_proportionate'] >= 1.125]
# above10c = backtest_test[backtest_test['model_edge_proportionate'] >= 1.25]
# above15c = backtest_test[backtest_test['model_edge_proportionate'] >= 1.375]
# above20c = backtest_test[backtest_test['model_edge_proportionate'] >= 1.5]
# above30c = backtest_test[backtest_test['model_edge_proportionate'] >= 1.75]

# # Calculate evolutions of profits for each threshold

# dfall = all_bets
# dfall['cumulative_profit'] = dfall['linear_target'].cumsum()

# df0c = above0c
# df0c['cumulative_profit'] = df0c['linear_target'].cumsum()

# df5c = above5c
# df5c['cumulative_profit'] = df5c['linear_target'].cumsum()

# df10c = above10c
# df10c['cumulative_profit'] = df10c['linear_target'].cumsum()

# df15c = above15c
# df15c['cumulative_profit'] = df15c['linear_target'].cumsum()

# df20c = above20c
# df20c['cumulative_profit'] = df20c['linear_target'].cumsum()

# df30c = above30c
# df30c['cumulative_profit'] = df30c['linear_target'].cumsum()

In [ ]:
# # Create dataframes of horses above specific proba_difference thresholds

# all_bets = backtest_test
# above0d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1]
# above5d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1.05]
# above10d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1.1]
# above15d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1.15]
# above20d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1.2]
# above30d = backtest_test[backtest_test['model_edge_prop_scaled'] >= 1.25]

# # Calculate evolutions of profits for each threshold

# dfall = all_bets
# dfall['cumulative_profit'] = dfall['linear_target'].cumsum()

# df0d = above0d
# df0d['cumulative_profit'] = df0c['linear_target'].cumsum()

# df5d = above5d
# df5d['cumulative_profit'] = df5c['linear_target'].cumsum()

# df10d = above10d
# df10d['cumulative_profit'] = df10c['linear_target'].cumsum()

# df15d = above15d
# df15d['cumulative_profit'] = df15c['linear_target'].cumsum()

# df20d = above20d
# df20d['cumulative_profit'] = df20c['linear_target'].cumsum()

# df30d = above30d
# df30d['cumulative_profit'] = df30c['linear_target'].cumsum()

In [ ]:
# import matplotlib.pyplot as plt
# # Plot
# plt.figure(figsize=(10,6))
# #plt.plot(range(1, len(dfall) + 1), dfall['cumulative_profit'], label='all bets')
# #plt.plot(range(1, len(df0d) + 1), df0d['cumulative_profit'], label='1 threshold')
# #plt.plot(range(1, len(df5d) + 1), df5d['cumulative_profit'], label='1.125 threshold')
# plt.plot(range(1, len(df10d) + 1), df10d['cumulative_profit'], label='1.25 threshold')
# plt.plot(range(1, len(df15d) + 1), df15d['cumulative_profit'], label='1.375 threshold')
# plt.plot(range(1, len(df20d) + 1), df20d['cumulative_profit'], label='1.5 threshold')
# plt.plot(range(1, len(df30d) + 1), df30d['cumulative_profit'], label='1.75 threshold')
# #plt.plot(range(1, len(df5c) + 1), df5c['cumulative_profit'], label='1.05 threshold unscaled')
# #plt.plot(range(1, len(df10c) + 1), df10c['cumulative_profit'], label='1.1 threshold unscaled')
# #plt.plot(range(1, len(df15c) + 1), df15c['cumulative_profit'], label='1.15 threshold unscaled')
# #plt.plot(range(1, len(df20c) + 1), df20c['cumulative_profit'], label='1.2 threshold unscaled')
# #plt.plot(range(1, len(df30c) + 1), df30c['cumulative_profit'], label='1.25 threshold unscaled')

# plt.title("xxx")
# plt.xlabel("number of bets")
# plt.ylabel("profit (£)")
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()